In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel, LdaMulticore, TfidfModel
from gensim.corpora import Dictionary
from nltk.tokenize import RegexpTokenizer
from datetime import datetime
from gensim.models import KeyedVectors
import nltk
import pandas as pd
import re
import math
import spacy
import time
import os
import shutil
import gensim
import string
import sys
import re

In [3]:
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])
def preprocessing(raw_text):
    # lowercasing, stemming and removing stop words
    sw = stopwords.words('english')
    sw2 = ['be', 'go', 'can', 'also', 'just', 'would', 'could']
    eng_re = re.compile(r'^[a-zA-Z]*$')
    words = []
    for w in raw_text.lower().split():
        # remove stop words
        if not w in sw:
            # remove non-ascii characters
            s = re.sub(r'[^\x00-\x7f]', r'', w)
            ss = re.sub(r'https?:\/\/.*', r'', s)
            if ss != '':
                words.append(ss)
    doc = nlp(" ".join(words))
    words_lemmatized = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']]
    # remove some specific words, like "breast", "cancer", "#"
    words_lemmatized2 = [x for x in words_lemmatized if (x not in ["#", "-", "breast", "cancer", "breastcancer"]) and 
                                                        (len(x) > 1) and 
                                                        (re.match(eng_re, x)) and
                                                        (x not in sw2)]
    return " ".join(words_lemmatized2)

In [4]:
path1 = "/Users/yhu245/Documents/GitHub/breast_cancer_analysis/" #path saving unlabeled data
path2 = "/Users/yhu245/Dropbox/CS584-textmining/dwr4xn8kcv-3/" #path saving word2vec model
df_tweets = pd.read_csv(path1 + "BreastCancer_Rawdata_TWs_unlabeled_predicted.csv")
df_reddit = pd.read_csv(path1 + "q4_tw_rd/data/data_reddit_processed.csv")

In [5]:
def clean_data(df=df_tweets, tweet_col='text'):
    
    df_copy = df.copy()
    
    # drop rows with empty values
    df_copy.dropna(inplace=True)
    
    
    # lower the tweets
    df_copy['preprocessed_' + tweet_col] = df_copy[tweet_col].str.lower()
    
    # filter out stop words and URLs
    df_copy['preprocessed_' + tweet_col] = df_copy['preprocessed_' + tweet_col].apply(lambda row: preprocessing(row))
    #df_copy['preprocessed_' + tweet_col] = df_copy['preprocessed_' + tweet_col].apply(lambda row: tokenize(row))
    #tokenizer = RegexpTokenizer(r"\w+")
    #df_copy['tokenized_' + tweet_col] = df_copy['preprocessed_' + tweet_col].apply(lambda row: tokenizer.tokenize(row))
    #df_copy['preprocessed_' + tweet_col] = df_copy['tokenized_text'].apply(lambda row: ' '.join(row))
   
    return df_copy
  

## Reddti Data Analysis

In [6]:
df_reddits_clean = clean_data(df_reddit)
df_reddits_clean.head()

,user_id,timestamp,text,preprocessed_text
0,incredally07,1602699352,I just need to complain a little. I got my la...,need complain little get last tissue expander ...
1,cookiebatter123,1602637885,My mum (50) has just been diagnosed with tripl...,diagnose triple negative doctor say next month...
2,wonkishgardener,1602630055,3 days of radiation left (woo hoo!) and sudden...,day radiation left woo suddenly almost overnig...
3,HonorableMention111,1602628724,I’ve been on chemo since the beginning of June...,have chemo begin final tomorrow think finally ...
4,Handy_Raccoon323,1602627408,Please excuse the formatting I'm on shitty mob...,excuse format shitty hubby doctor know punch b...


In [8]:
document_reddit = df_reddits_clean['preprocessed_text'].tolist()

In [9]:
%%time
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(document_reddit)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_reddit_tfidf = pd.DataFrame(denselist, columns=feature_names)

CPU times: user 3.55 s, sys: 463 ms, total: 4.01 s
Wall time: 4.03 s


In [10]:
df_reddit_tfidf.head()

,aaahhhhh,aback,abandon,abate,abcess,abdomen,abdominal,abemaciclib,ability,ablation,...,ysc,yup,zap,zip,zoladex,zoloft,zombie,zometa,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
reddit_top = pd.DataFrame({'mean': df_reddit_tfidf.mean(), 'name': df_reddit_tfidf.columns}).sort_values('mean', ascending = False)

In [12]:
reddit_top[:50]

,mean,name
feel,0.033532,feel
know,0.031709,know
get,0.031209,get
say,0.026339,say
week,0.025014,week
year,0.024566,year
lump,0.024091,lump
doctor,0.023975,doctor
find,0.023475,find
want,0.023171,want


## Twitter Data Analysis

In [46]:
idx = []
k = 0
for sen in df_tweets['text']:
    if 'My name is fiza' in sen:
        idx.append(k)
    k += 1

idx = idx[1:]
for i in idx:
    df_tweets = df_tweets.drop(i)    

In [48]:
%%time
df_tweets_clean = clean_data(df_tweets)
df_tweets_clean.head()

CPU times: user 5min 1s, sys: 8.55 s, total: 5min 10s
Wall time: 5min 13s


,tweet_id,text,class,preprocessed_text
0,1314367341078228992,@lscreeden @ChicagoBears @BearsLocal @BearsOut...,0.0,guy beautiful wife battle right now
1,1314371953906479104,🎀 | PINKtober is HERE &amp; Jessup 🥎 can’t wai...,0.0,wait new year long grab one percentage proceed...
2,1314426946348957696,@HeelJonJones @Kuckaneer He’s talking about iz...,0.0,talk tit
3,1314333474141151232,Join me and help change the future of breast c...,0.0,join change future
4,1314332965216882688,Sarah Harding is ‘holding up well’ with mum Ma...,0.0,hold care


In [49]:
document_tweets = df_tweets_clean['preprocessed_text'].tolist()

In [50]:
document_tweets = [x.split() for x in document_tweets]
dct = Dictionary(document_tweets)  # fit dictionary
corpus = [dct.doc2bow(line) for line in document_tweets]  # convert corpus to BoW format
model = TfidfModel(corpus)  # fit model
tfidf = model[corpus]
tfidf_all = []
for i in tfidf:
    tfidf_all.append(i)

In [54]:
from collections import defaultdict 

In [55]:
d = defaultdict()
for word_num in range(len(dct)):
    if word_num % 5000 == 0:
        print(word_num)
    for sent in tfidf_all:
            temp = [word for word in sent if word[0] == word_num]
            if temp != [] and word_num in d:
                d[word_num] += temp[0][1]
            if word_num not in d:
                d[word_num] = 0

0
5000
10000
15000
20000
25000
30000
35000
40000


In [56]:
for key, value in d.items():
        d[key] = value/len(tfidf)
sss = []
for word, idnum in dct.token2id.items():
    sss.append([word, d[idnum]])

In [57]:
tweets_top = pd.DataFrame(sss, columns = ['name', 'mean']).sort_values('mean', ascending = False)

In [58]:
tweets_top[:50]

,name,mean
41,woman,0.016086
15,year,0.013334
192,help,0.013204
53,awareness,0.012889
123,get,0.012667
77,month,0.011511
288,say,0.011294
219,know,0.010766
166,treatment,0.010378
234,die,0.009502


In [59]:
tweets_top.iloc[:50, 0]

41         woman
15          year
192         help
53     awareness
123          get
77         month
288          say
219         know
166    treatment
234          die
221          mom
34          risk
380       people
49       patient
62          pink
356         life
122         find
330         make
423     diagnose
39        matter
79         fight
226        stage
339          man
230     survivor
186        thank
115         good
300         need
85          time
164         take
9            new
13      research
121        early
82        mother
879        think
64           see
253        today
583         cell
361        check
499        study
212          day
165         tell
28        family
153         live
196      surgery
30        friend
453          amp
728         lose
167         want
421          use
125         love
Name: name, dtype: object

### Twitter Data Analysis Class 1

In [68]:
%%time
document_tweets_1 = df_tweets_clean.loc[df_tweets_clean['class']==1, 'preprocessed_text'].tolist()
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(document_tweets_1)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df_tweets_tfidf_1 = pd.DataFrame(denselist, columns=feature_names)

CPU times: user 56.5 s, sys: 31.6 s, total: 1min 28s
Wall time: 1min 55s


In [69]:
tweets_top_1 = pd.DataFrame({'mean': df_tweets_tfidf_1.mean(), 'name': df_tweets_tfidf_1.columns}).sort_values('mean', ascending = False)

In [70]:
tweets_top_1[:50]

,mean,name
year,0.032388,year
get,0.022207,get
survivor,0.018603,survivor
stage,0.017863,stage
treatment,0.017130,treatment
surgery,0.015366,surgery
know,0.015325,know
help,0.015079,help
thank,0.014902,thank
life,0.014709,life


### Twitter Data Analysis Class 0

In [60]:
document_tweets_0 = df_tweets_clean.loc[df_tweets_clean['class']==0, 'preprocessed_text'].tolist()

In [61]:
document_tweets_0 = [x.split() for x in document_tweets_0]

In [62]:
dct0 = Dictionary(document_tweets_0)  # fit dictionary
corpus0 = [dct0.doc2bow(line) for line in document_tweets_0]  # convert corpus to BoW format
model0 = TfidfModel(corpus0)  # fit model
tfidf0 = model0[corpus0]
tfidf_all0 = []
for i in tfidf0:
    tfidf_all0.append(i)

In [63]:
d0 = defaultdict()
for word_num in range(len(dct0.token2id)):
    if word_num%5000 == 0:
        print(word_num)
    for sent in tfidf_all0:
            temp = [word for word in sent if word[0] == word_num]
            if temp != [] and word_num in d0:
                d0[word_num] += temp[0][1]
            if word_num not in d0:
                d0[word_num] = 0

0
5000
10000
15000
20000
25000
30000
35000
40000


In [64]:
for key, value in d0.items():
        d0[key] = value/len(tfidf_all0)
sss0 = []
for word, idnum in dct0.token2id.items():
    sss0.append([word, d0[idnum]])

In [65]:
tweets_top_0 = pd.DataFrame(sss0, columns = ['name', 'mean']).sort_values('mean', ascending = False)

In [66]:
tweets_top_0[:50]

,name,mean
41,woman,0.016860
53,awareness,0.013797
184,help,0.013331
123,get,0.012064
77,month,0.011803
15,year,0.011683
271,say,0.011527
211,know,0.010559
405,treatment,0.009984
226,die,0.009831


In [67]:
tweets_top_0.loc[:50, "name"]

41               woman
53           awareness
184               help
123                get
77               month
15                year
271                say
211               know
405          treatment
226                die
213                mom
34                risk
62                pink
49             patient
364             people
39              matter
122               find
323                man
313               make
340               life
408           diagnose
79               fight
9                  new
13            research
82              mother
572               cell
283               need
121              early
115               good
178              thank
             ...      
2277             sound
2802           mention
149              claim
613              virus
722             choose
844            protect
400              power
2557          chemical
209                hit
1457           comment
160               bear
3914              dose
983        